In [28]:
from docx import Document
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import time
import re

In [29]:
# Создаем регулярные выражения
name_table_re = re. compile(r'Наименование государственной услуги:.*?(Реализация\s.+?)[2]')
cat_table_re = re.compile(r'Категории потребителей государственной услуги:.+?Физические лица,(.+?)[3]')

doc = Document('БРИТ.docx')
# последовательность всех таблиц документа
all_tables = doc.tables
print('Всего таблиц в документе:', len(all_tables))

# создаем пустой словарь под данные таблиц
data_tables = {i:None for i in range(len(all_tables))}
# проходимся по таблицам

lst_df =[]
for i, table in enumerate(all_tables):

    # print('\nДанные таблицы №', i)
    # создаем список строк для таблицы `i` (пока пустые)
    data_tables[i] = [[] for _ in range(len(table.rows))]
    # проходимся по строкам таблицы `i`
    for j, row in enumerate(table.rows):
        # проходимся по ячейкам таблицы `i` и строки `j`
        for cell in row.cells:
            # добавляем значение ячейки в соответствующий
            # список, созданного словаря под данные таблиц
            data_tables[i][j].append(cell.text)

    # смотрим извлеченные данные
    # (по строкам) для таблицы `i`
    df = pd.DataFrame(data_tables[i])
    lst_df.append(df)
dct_data = dict()

# Извлекаем
for i in range(1,len(lst_df),3):
    print(i)
    # Получаем совокупный текст таблиц с названием и категорией студентов
    text = lst_df[i].sum().sum()
    # Удаляем символы переноса
    text = text.replace('\n','')

    # Ищем название таблицы
    match_name = re.search(name_table_re,text)
    name_table = match_name.group(1).strip()
    # print(f'Название таблицы {name_table}')
    # Ищем категорию таблицы
    match_cat = re.search(cat_table_re,text)
    name_cat = match_cat.group(1).strip()
    # print(f'Категория таблицы {name_cat}')
    # print('*********************')

    # dct_data[name_table] ={name_cat:''}
    if name_table not in dct_data:
        dct_data[name_table]={name_cat:''}
    else:
        dct_data[name_table].update({name_cat:''})
    if i == 1:
        # удаляем первые 4 строки в датафрейме
        df =  lst_df[3].drop(labels=[0,1,2,3],axis=0)
        # избавляемся от знаков переноса строки

        df = df.applymap(lambda x: x.replace('\n', ''))

    else:
        # удаляем первые 4 строки в датафрейме
        df = lst_df[i+2].drop(labels=[0,1,2,3],axis=0)
        # избавляемся от знаков переноса строки
        df = df.applymap(lambda x:x.replace('\n',''))
    #Добавляем датафрейм в словарь
    dct_data[name_table][name_cat] = df

Всего таблиц в документе: 22
1
4
7
10
13
16
19


In [31]:
dct_data.keys()

dict_keys(['Реализация основных профессиональных образовательных программ среднего профессионального образования - программ подготовки квалифицированных рабочих, служащих', 'Реализация образовательных программ среднего профессионального образования - программ подготовки квалифицированных рабочих, служащих', 'Реализация образовательных программ среднего профессионального образования - программ подготовки специалистов среднего звена', 'Реализация основных профессиональных образовательных программ профессионального обучения - программ профессиональной подготовки по профессиям рабочих, должностям служащих', 'Реализация дополнительных профессиональных программ повышения квалификации'])

In [73]:
one=dct_data['Реализация основных профессиональных образовательных программ среднего профессионального образования - программ подготовки квалифицированных рабочих, служащих']['имеющие основное общее образование']

In [109]:
one_d = one.drop([0,2,3,4,5,6,7,8],axis=1)

In [110]:
one_d

,1,9,10,11,12,13,14,15,16
4,08.01.18 Электромонтажник электрических сетей ...,46,44,"5,0",2,,,,
5,15.01.05 Сварщик ручной и частично механизиров...,139,137,"5,0",7,,,,
6,15.01.26 Токарь-универсал,19,18,"5,0",1,,,,
7,15.01.33 Токарь на станках с числовым программ...,72,72,"5,0",4,,,,
8,15.01.35 Мастер слесарных работ,71,61,"5,0",4,"8,4",6,"5 перевод, 1 псж",
9,"43.01.09 Повар, кондитер",58,59,"5,0",3,,,,
10,23.01.03 Автомеханик,47,47,"5,0",2,,,,
11,23.01.09 Машинист локомотива,385,382,"5,0",19,,,,
12,23.01.10 Слесарь по обслуживанию и ремонту под...,145,137,"5,0",7,"5,5",1,перевод,
13,23.01.17 Мастер по ремонту и обслуживанию авто...,76,73,"5,0",4,,,,


In [105]:
one_d.insert(0,'Название','Реализация основных профессиональных образовательных программ среднего профессионального образования - программ подготовки квалифицированных рабочих, служащих')

In [106]:
one_d.insert(1,'Категория','имеющие основное общее образование')

In [107]:
one_d

,Название,Категория,1,9,10,11,12,13,14,15
4,Реализация основных профессиональных образоват...,имеющие основное общее образование,08.01.18 Электромонтажник электрических сетей ...,46,44,"5,0",2,,,
5,Реализация основных профессиональных образоват...,имеющие основное общее образование,15.01.05 Сварщик ручной и частично механизиров...,139,137,"5,0",7,,,
6,Реализация основных профессиональных образоват...,имеющие основное общее образование,15.01.26 Токарь-универсал,19,18,"5,0",1,,,
7,Реализация основных профессиональных образоват...,имеющие основное общее образование,15.01.33 Токарь на станках с числовым программ...,72,72,"5,0",4,,,
8,Реализация основных профессиональных образоват...,имеющие основное общее образование,15.01.35 Мастер слесарных работ,71,61,"5,0",4,"8,4",6,"5 перевод, 1 псж"
9,Реализация основных профессиональных образоват...,имеющие основное общее образование,"43.01.09 Повар, кондитер",58,59,"5,0",3,,,
10,Реализация основных профессиональных образоват...,имеющие основное общее образование,23.01.03 Автомеханик,47,47,"5,0",2,,,
11,Реализация основных профессиональных образоват...,имеющие основное общее образование,23.01.09 Машинист локомотива,385,382,"5,0",19,,,
12,Реализация основных профессиональных образоват...,имеющие основное общее образование,23.01.10 Слесарь по обслуживанию и ремонту под...,145,137,"5,0",7,"5,5",1,перевод
13,Реализация основных профессиональных образоват...,имеющие основное общее образование,23.01.17 Мастер по ремонту и обслуживанию авто...,76,73,"5,0",4,,,


In [99]:
one_d.to_excel('Тест.xlsx',index=False)